In [333]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot
import xgboost as xgb
import numpy as np

In [334]:
import os
path = os.path.dirname(xgb.__file__)
print(path)

/home/agavrilenko/anaconda3/lib/python3.6/site-packages/xgboost


In [335]:
#help(xgboost.training.train)
#help(xgb.train)

In [336]:
# load data
dataset = loadtxt('data/data.txt', delimiter=",")
# split data into X and y
X = dataset[:,0:8]
Y = dataset[:,8]
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=177)

In [337]:
dtrain = xgb.DMatrix(data = X_train, label = y_train, silent= True)

deval = xgb.DMatrix(data = X_test, label = y_test, silent= True)
# evals_list = []
# evals_list.append((deval, "eval matrix"))
num_epochs = 200
params = {}
params['num_boost_round'] = num_epochs
params['early_stopping_rounds'] = 1000
#params['verbose_eval'] = 1
params['eval_metric'] = 'logloss'
params['evals_result'] = {}
params['max_depth'] = 3

params['subsample'] = 1


In [343]:
learning_rate_previous = 0.1
cooldown_counter = 0
best = 100
progress = dict()
wait = 0 

def reduceLRonPlateau(i,n):

    factor = 0.3
    min_lr = 1e-30
    min_delta = 1e-4
    patience = 3
    verbose = 0
    cooldown = 0    
    def in_cooldown():
            return cooldown_counter > 0
    
    def _reset():
        cooldown_counter = cooldown
        wait = 0
        return wait, cooldown_counter
    
    def checker(): 
        global wait
        global best
        global learning_rate_previous
        global cooldown_counter
        new_learning_rate = learning_rate_previous
        
        if (i == 0):
            wait, cooldown_counter = _reset()
            print("Initialization...")           
            
        if (i>0):
            previous_loss = progress['eval']['logloss'][i-1]
            if in_cooldown():
                cooldown_counter -= 1
                wait = 0
            
            else:
                if (previous_loss < best- min_delta):
                    best= previous_loss
                    wait = 0
                
                else:
                    wait+=1
                    if wait > patience:
                        if old_lr > min_lr:
                            new_learning_rate = learning_rate_previous * factor
                            new_learning_rate = max(new_learning_rate, min_lr)
                           
                            cooldown_counter = cooldown
                            wait = 0
                        
        return new_learning_rate

   
    new_learning_rate = checker()
    global learning_rate_previous
    #print("iter: ", i," old_lr: " ,learning_rate_previous, " new_lr: ", new_learning_rate)
    
    learning_rate_previous = new_learning_rate
        
    return 0.1

In [347]:

def run_gbm(dtrain, dvalid, param, evals_result, learning_rate):
    # check training arguments in param
    n_round = param.get('num_boost_round', 100)
    early_stop = param.get('early_stopping_rounds', 0)
    verbose_eval = param.get('verbose_eval', 0)
    # specify validations set to watch performance
    watchlist = [(dtrain,'train') ,(deval,'eval')]
    #callbacks_list = [learning_rates]
    
    bst = xgb.train(params=param,
            dtrain=dtrain,
            num_boost_round=n_round,
            evals=watchlist,
            early_stopping_rounds=early_stop,
            verbose_eval = verbose_eval,
            evals_result = evals_result, 
            callbacks = [xgb.callback.reset_learning_rate(reduceLRonPlateau)])
    print("done")                  
    return bst 

In [348]:
model1 = run_gbm(dtrain, deval, params,progress, learning_rate = 0.1)

Initialization...
done


In [349]:
# fit model no training data

model = XGBClassifier(n_estimators=num_epochs, learning_rate=0.1, subsample=1, eta = 0.2 )

eval_set = [(X_train, y_train), (X_test, y_test)]
model.fit(X_train, y_train, eval_metric=["error", "logloss"], eval_set=eval_set, verbose=False,)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.2, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=200, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [342]:
# # retrieve performance metrics
# results = model.evals_result()
# #results1 = model1.evals_result()
# epochs = len(results['validation_0']['error'])
# x_axis = range(0, 200)

# # plot log loss
# fig, ax = pyplot.subplots(figsize=(10, 6))
# ax.plot(x_axis, results['validation_0']['logloss'][:200:], label='Train')
# ax.plot(x_axis, results['validation_1']['logloss'][:200:], label='Eval')
# ax.plot(x_axis, progress['train']['logloss'][:200:], label = 'customTrain')
# ax.plot(x_axis, progress['eval']['logloss'][:200:], label = 'customEval')
# ax.legend()
# pyplot.ylabel('Log Loss')
# pyplot.title('XGBoost Log Loss')

# pyplot.show()
# print("iter: ", progress['eval']['logloss'][:200:].index(min(progress['eval']['logloss'][:200:])), " min customEval:", min(progress['eval']['logloss'][:100:]))
# print("iter: ", results['validation_1']['logloss'][:200:].index(min(results['validation_1']['logloss'][:200:])), " min Eval:", min(results['validation_1']['logloss'][:100:]))